#**Text Classification by Fine-tuning Language Model**
##**1. Data Loading**

In [ ]:
# Install simpletransformers package
!pip install simpletransformers

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset (replace with your dataset path)
data = pd.read_csv('NLP Dataset.csv')

# Rename columns to match the expected format
data = data.rename(columns={'INPUT': 'text', 'INTENT': 'label'})

# Exploratory Data Analysis (EDA)
print(data.info())  # Overview of data structure
print(data['label'].value_counts())  # Class distribution

# Split dataset into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.3, random_state=42)

# Preparing the data in the correct format for SimpleTransformers
train_df = pd.DataFrame({
    'text': train_data['text'],
    'labels': train_data['label']
})

val_df = pd.DataFrame({
    'text': val_data['text'],
    'labels': val_data['label']
})

# Display the first few rows of the training and validation data
print("Training Data:")
print(train_df.head())

print("\nValidation Data:")
print(val_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2484 non-null   object
 1   label   2484 non-null   object
dtypes: object(2)
memory usage: 38.9+ KB
None
label
cancel_order                92
change_order                92
change_shipping_address     92
check_cancellation_fee      92
check_invoice               92
check_payment_methods       92
check_refund_policy         92
complaint                   92
contact_customer_service    92
contact_human_agent         92
create_account              92
delete_account              92
delivery_options            92
delivery_period             92
edit_account                92
get_invoice                 92
get_refund                  92
newsletter_subscription     92
payment_issue               92
place_order                 92
recover_password            92
registration_problems       92
review          

##**2. Text Preprocessing**

In [ ]:
import re

# Define a function to clean text data
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = text.strip()

    return text

# Apply the cleaning function to the dataset
train_df['text'] = train_df['text'].apply(clean_text)
val_df['text'] = val_df['text'].apply(clean_text)

# Display the first few rows of the cleaned training data
print("Cleaned Training Data:")
print(train_df.head())

# Display the first few rows of the cleaned validation data
print("\nCleaned Validation Data:")
print(val_df.head())

Cleaned Training Data:
                                                   text  \
1618  id like to unsubscrie to the corporate newslet...   
309   i cannot find the withdrawal penalty could you...   
591   want help to see in what cases can i ask for r...   
1957  i do not know what i need to do to inform of s...   
932   where could i create a standard account for my...   

                       labels  
1618  newsletter_subscription  
309    check_cancellation_fee  
591       check_refund_policy  
1957    registration_problems  
932            create_account  

Cleaned Validation Data:
                                                   text  \
420   what do i need to do to give a quick look at i...   
1309  i need assistance to edit the personal details...   
2023  report registration problem i have been facing...   
1360  modify information on gold account this order ...   
2186  i have got to enter a new shipping address hel...   

                       labels  
420             c

##**3. Text Embedding using BERT and RoBERTa**

In [ ]:
from simpletransformers.classification import ClassificationModel

# Get the number of unique labels (intents) in the dataset
num_labels = len(data['label'].unique())

# Create a BERT model for text classification
bert_model = ClassificationModel(
    'bert',
    'bert-base-uncased',
    num_labels=num_labels,
    use_cuda=False  # Enable GPU if available
)

# Create a RoBERTa model for text classification
roberta_model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=num_labels,
    use_cuda=False  # Enable GPU if available
)

print("BERT and RoBERTa models initialized successfully!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

BERT and RoBERTa models initialized successfully!


##**4. Model Training with BERT and RoBERTa**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from simpletransformers.classification import ClassificationArgs

# Convert string labels to integer labels using LabelEncoder
label_encoder = LabelEncoder()
train_df['labels'] = label_encoder.fit_transform(train_df['labels'])
val_df['labels'] = label_encoder.transform(val_df['labels'])

# Set up model arguments with custom hyperparameters
model_args = ClassificationArgs(
    num_train_epochs=3,       # Start with 3 epochs
    train_batch_size=8,       # Use a batch size of 8
    eval_batch_size=8,        # Same for evaluation
    learning_rate=3e-5,       # Learning rate
    max_seq_length=128,       # Max sequence length
    weight_decay=0.01,        # Weight decay
    warmup_steps=0,           # Optional: adjust based on total steps
    logging_steps=50,         # Log training progress every 50 steps
    save_steps=200,           # Save the model every 200 steps
    overwrite_output_dir=True,  # Overwrite the output directory
    output_dir='outputs',     # Directory to save model outputs
)

# Train the BERT model with custom hyperparameters
bert_model = ClassificationModel(
    'bert',
    'bert-base-uncased',
    num_labels=num_labels,
    args=model_args,
    use_cuda=False  # Set to True if using GPU
)
bert_model.train_model(train_df)

# Train the RoBERTa model with custom hyperparameters
roberta_model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=num_labels,
    args=model_args,
    use_cuda=False  # Set to True if using GPU
)
roberta_model.train_model(train_df)

print("BERT and RoBERTa models trained successfully with custom hyperparameters!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/218 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/218 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/218 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/218 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/218 [00:00<?, ?it/s]

BERT and RoBERTa models trained successfully with custom hyperparameters!


##**5. Evaluation on Validation Set**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate BERT on validation data
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

# Decode predictions back to original labels
bert_predictions = np.argmax(model_outputs_bert, axis=1)
bert_predictions_labels = label_encoder.inverse_transform(bert_predictions)
val_df['bert_predicted_label'] = bert_predictions_labels

# Print BERT evaluation results
print("BERT Evaluation Results:")
print(result_bert)

# Classification report for BERT
print("\nBERT Classification Report:")
print(classification_report(val_df['labels'], bert_predictions, target_names=label_encoder.classes_))

# Evaluate RoBERTa on validation data
result_roberta, model_outputs_roberta, wrong_predictions_roberta = roberta_model.eval_model(val_df)

# Decode predictions back to original labels
roberta_predictions = np.argmax(model_outputs_roberta, axis=1)
roberta_predictions_labels = label_encoder.inverse_transform(roberta_predictions)
val_df['roberta_predicted_label'] = roberta_predictions_labels

# Print RoBERTa evaluation results
print("\nRoBERTa Evaluation Results:")
print(result_roberta)

# Classification report for RoBERTa
print("\nRoBERTa Classification Report:")
print(classification_report(val_df['labels'], roberta_predictions, target_names=label_encoder.classes_))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/94 [00:00<?, ?it/s]

BERT Evaluation Results:
{'mcc': np.float64(0.9694164205284739), 'eval_loss': 0.31620499618509984}

BERT Classification Report:
                          precision    recall  f1-score   support

            cancel_order       1.00      0.95      0.97        19
            change_order       0.97      1.00      0.98        28
 change_shipping_address       0.97      1.00      0.98        29
  check_cancellation_fee       0.97      1.00      0.99        36
           check_invoice       0.90      0.90      0.90        29
   check_payment_methods       1.00      1.00      1.00        30
     check_refund_policy       1.00      0.96      0.98        28
               complaint       1.00      1.00      1.00        20
contact_customer_service       1.00      1.00      1.00        29
     contact_human_agent       1.00      1.00      1.00        28
          create_account       0.92      1.00      0.96        22
          delete_account       0.97      0.88      0.92        34
        deliv

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/94 [00:00<?, ?it/s]


RoBERTa Evaluation Results:
{'mcc': np.float64(0.988889244859663), 'eval_loss': 0.06457568774435749}

RoBERTa Classification Report:
                          precision    recall  f1-score   support

            cancel_order       1.00      1.00      1.00        19
            change_order       1.00      1.00      1.00        28
 change_shipping_address       0.97      1.00      0.98        29
  check_cancellation_fee       1.00      1.00      1.00        36
           check_invoice       0.91      1.00      0.95        29
   check_payment_methods       1.00      1.00      1.00        30
     check_refund_policy       1.00      1.00      1.00        28
               complaint       1.00      1.00      1.00        20
contact_customer_service       1.00      1.00      1.00        29
     contact_human_agent       1.00      1.00      1.00        28
          create_account       0.88      1.00      0.94        22
          delete_account       1.00      0.94      0.97        34
       

In [ ]:
import pandas as pd

# Create a dictionary with the table data for BERT and RoBERTa
data = {
    "No.": [1, 2],
    "Model Name": ["BERT", "RoBERTa"],
    "Precision": [0.97, 0.99],  # Macro avg precision from classification reports
    "Recall": [0.97, 0.99],     # Macro avg recall from classification reports
    "F1 Score": [0.97, 0.99],   # Macro avg F1-score from classification reports
    "Accuracy": [0.97, 0.99],   # Accuracy from classification reports
    "MCC": [0.969, 0.989]       # MCC from evaluation results
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

# Display the table
df

,No.,Model Name,Precision,Recall,F1 Score,Accuracy,MCC
0,1,BERT,0.97,0.97,0.97,0.97,0.969
1,2,RoBERTa,0.99,0.99,0.99,0.99,0.989


##**6. Saving the Model**

In [ ]:
# Save the BERT model manually
bert_model.model.save_pretrained("bert_model")
bert_model.tokenizer.save_pretrained("bert_model")
print("BERT model saved manually!")
# Save the RoBERTa model manually
roberta_model.model.save_pretrained("roberta_model")
roberta_model.tokenizer.save_pretrained("roberta_model")
print("RoBERTa model saved manually!")

BERT model saved manually!
RoBERTa model saved manually!


##**7. Prediction on Real-World Input**

In [ ]:
# Load the saved BERT model
bert_model = ClassificationModel('bert', 'bert_model', use_cuda=False)

# Real-world input text (aligned with your dataset's context)
real_world_text = [
    "I need to cancel my order {{Order Number}}.",
    "How can I change the shipping address for my order?",
    "What is the cancellation fee for my order?"
]

# Predict the class using BERT
predictions_bert, _ = bert_model.predict(real_world_text)

# Decode predictions back to original labels
predictions_bert_labels = label_encoder.inverse_transform(predictions_bert)

# Print BERT predictions
print("BERT Predictions:")
for text, pred_label in zip(real_world_text, predictions_bert_labels):
    print(f"Text: {text} -> Predicted Intent: {pred_label}")

# Load the saved RoBERTa model
roberta_model = ClassificationModel('roberta', 'roberta_model', use_cuda=False)

# Predict the class using RoBERTa
predictions_roberta, _ = roberta_model.predict(real_world_text)

# Decode predictions back to original labels
predictions_roberta_labels = label_encoder.inverse_transform(predictions_roberta)

# Print RoBERTa predictions
print("\nRoBERTa Predictions:")
for text, pred_label in zip(real_world_text, predictions_roberta_labels):
    print(f"Text: {text} -> Predicted Intent: {pred_label}")

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

BERT Predictions:
Text: I need to cancel my order {{Order Number}}. -> Predicted Intent: change_order
Text: How can I change the shipping address for my order? -> Predicted Intent: change_shipping_address
Text: What is the cancellation fee for my order? -> Predicted Intent: check_cancellation_fee


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


RoBERTa Predictions:
Text: I need to cancel my order {{Order Number}}. -> Predicted Intent: cancel_order
Text: How can I change the shipping address for my order? -> Predicted Intent: change_shipping_address
Text: What is the cancellation fee for my order? -> Predicted Intent: cancel_order


#**8. Analysis**
### Discussion of Results

1. **BERT**:
   - **Performance**: Achieved an MCC of 0.969 and an accuracy of 0.97. The classification report shows high precision, recall, and F1-scores across most classes.
   - **Analysis**: BERT performed exceptionally well due to its ability to capture contextual information, making it highly effective for text classification tasks.

2. **RoBERTa**:
   - **Performance**: Outperformed BERT with an MCC of 0.989 and an accuracy of 0.99. The classification report shows near-perfect precision, recall, and F1-scores across all classes.
   - **Analysis**: RoBERTa, an optimized version of BERT, performed even better, likely due to its improved training methodology and larger dataset.


### Best Performing Feature Set

- **Transformer Models (BERT and RoBERTa)**: These models outperformed traditional NLP features (BoW, TF-IDF, FastText) by a significant margin. This is because transformer models capture deep contextual relationships in text, which is crucial for understanding intent in customer queries.

### Challenges and Interesting Findings

- **Transformer Dominance**: BERT and RoBERTa significantly outperformed traditional models, highlighting the importance of contextual understanding in NLP tasks.
- **Class Imbalance**: Some classes had lower support, which could affect model performance. However, transformer models handled this well due to their robustness.
- **Training Time**: Transformer models require more computational resources and time compared to traditional models.

### Potential Improvements and Further Experiments

1. **Fine-Tuning**: Further fine-tune BERT and RoBERTa on domain-specific data to improve performance.
2. **Data Augmentation**: Use data augmentation techniques to balance class distribution and improve model generalization.
3. **Ensemble Methods**: Combine BERT/RoBERTa with other models to leverage their strengths.